# Deploying Strands Agents to AWS Lambda

This notebook demonstrates how to package Strands Agents SDK, build a Lambda layer, and deploy a Lambda function using Python and boto3.

If you're not familiar with the Strands Agents SDK, check out the [official documentation](https://strandsagents.com).

## Prerequisites

- Python 3.12 or later
- boto3 1.39 or later
- AWS CLI installed and configured with appropriate permissions

In addition, you should have the follwoing readily available:
- Amazon S3 bucket to package code
- AWS IAM role for automation access
- Knowledge bases are fully synched
- External functions for enterprise systems

## Step 0: Import required libraries

In [ ]:
import os
import json
import time
import boto3
import shutil
import zipfile
import tempfile
import subprocess
from pathlib import Path

## Step 1: Set required parameters

In [ ]:
Knowledge_Base_1_Id = '<provide 1st Knowledge Base Id>'
Knowledge_Base_2_Id = '<provide 2nd Knowledge Base Id>'
System_Function_1_Name = '<provide 1st System Function Name>'
System_Function_2_Name = '<provide 2nd System Function Name>'
Agent_Directory_Name = '<provide agent directory name based on event use case>'
CodeBucketForAutomationName = '<provide CodeBucketForAutomationName - not full ARN>'
SolutionAccessRoleArn = '<provide SolutionAccessRoleArn>'

## Step 2: Verify Lambda function code and requirements

We'll use the Lambda function code and requirements from the external files.

In [ ]:
# Verify Lambda function code exists
lambda_file = Path(f"{Agent_Directory_Name}/lambda_function.py")
if lambda_file.exists():
    print(f"Lambda function code found at {lambda_file}")
else:
    print(f"Lambda function code not found at {lambda_file}")
    
# Verify requirements file exists
requirements_file = Path(f"{Agent_Directory_Name}/requirements.txt")
if requirements_file.exists():
    print(f"Requirements file found at {requirements_file}")
else:
    print(f"Requirements file not found at {requirements_file}")

## Step 3: Deploy Lambda Layer with Minimal Dependencies

In [ ]:
def create_optimized_layer():
    '''Create a Lambda layer with all dependencies except excluded ones.'''
    with tempfile.TemporaryDirectory() as temp_dir:
        temp_path = Path(temp_dir)
        python_path = temp_path / "python"
        python_path.mkdir()
        
        # Define excluded packages (boto3 is pre-installed in Lambda)
        excludedlist = [
            # AWS SDK components (pre-installed in Lambda)
            "boto3", "botocore", "aws-requests-auth", "s3transfer", "jmespath",
            
            # Large packages with significant size
            "numpy", "pillow", "grpcio", "protobuf", "sqlalchemy", 
            "openai", "qdrant-client", "mem0ai", "slack-bolt", "slack_sdk",
            
            # Optional dependencies
            "pytz", "posthog", "opensearch-py", "sympy", "greenlet",
            
            # Web-related packages not needed in Lambda
            "uvicorn", "starlette", "httpx", "httpcore", "h2", "hpack",
            
            # Visualization and development tools
            "pygments", "tqdm", "markdown-it-py"
        ]
        
        # Install all requirements with their dependencies
        subprocess.run(
            ["pip", "install", "-r", str(requirements_file), "-t", str(python_path)],
            check=True
        )
        
        # Remove excluded packages if they were installed
        for pkg in excludedlist:
            pkg_path = python_path / pkg
            if pkg_path.exists() and pkg_path.is_dir():
                shutil.rmtree(pkg_path)
                print(f"Removed excluded package: {pkg}")
        
        # Create a zip file of the layer
        layer_zip_path = "strands_agents_minimal_layer.zip"
        with zipfile.ZipFile(layer_zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:
            for root, _, files in os.walk(temp_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, temp_path)
                    zipf.write(file_path, arcname)
        
        return layer_zip_path

# Create the Lambda layer
layer_zip_path = create_optimized_layer()
print(f"Lambda layer packaged at {layer_zip_path}")

# Upload the layer to AWS Lambda
def upload_layer_to_lambda(layer_zip_path):
    '''Upload the layer zip file to AWS Lambda and return the layer ARN.'''
    lambda_client = boto3.client('lambda')
    
    with open(layer_zip_path, 'rb') as zip_file:
        response = lambda_client.publish_layer_version(
            LayerName='StrandsAgentsLayer',
            Description='Strands Agents SDK and dependencies',
            Content={
                'ZipFile': zip_file.read()
            },
            CompatibleRuntimes=['python3.12']
        )
    
    return response['LayerVersionArn']

# Upload the layer to Lambda
layer_arn = upload_layer_to_lambda(layer_zip_path)
print(f"Lambda layer uploaded with ARN: {layer_arn}")

## Step 4: Package Lambda Function

In [ ]:
def package_lambda_function():
    '''Package the Lambda function code into a zip file.'''
    function_zip_path = "agent_lambda.zip"
    
    with zipfile.ZipFile(function_zip_path, "w", zipfile.ZIP_DEFLATED) as zipf:       
        # Add all Python files from the agent directory
        # We need to flatten the structure for Lambda to find the modules
        agent_dir = Path(Agent_Directory_Name)
        for file_path in agent_dir.glob("*.py"):
            zipf.write(file_path, file_path.name)
    
    return function_zip_path

# Package the Lambda function
function_zip_path = package_lambda_function()
print(f"Lambda function packaged at {function_zip_path}")

## Step 5: Deploy Lambda Function

In [ ]:
function_name = 'StrandsLambdaAgent'

def upload_to_s3(file_path, bucket_name, s3_key):
    '''Upload a file to S3 and return the S3 location.'''
    s3_client = boto3.client('s3')
    s3_client.upload_file(file_path, bucket_name, s3_key)
    return f"s3://{bucket_name}/{s3_key}"


def lambda_function_exists(function_name):
    client = boto3.client('lambda')
    try:
        client.get_function(FunctionName=function_name)
        return True
    except Exception as e:
        if e.response['Error']['Code'] == 'ResourceNotFoundException':
            return False
        else:
            raise


def deploy_lambda_function(function_zip_path, layer_arn, role_arn, bucket_name):
    '''Deploy the Lambda function to AWS using S3.'''
    # Upload the function zip to S3
    s3_key = f"lambda-functions/{os.path.basename(function_zip_path)}"
    s3_location = upload_to_s3(function_zip_path, bucket_name, s3_key)
    print(f"Function uploaded to {s3_location}")
    
    # Create the function from S3
    lambda_client = boto3.client('lambda')
    response = lambda_client.create_function(
        FunctionName=function_name,
        Runtime='python3.12',
        Role=role_arn,
        Handler='lambda_function.handler',
        Code={
            'S3Bucket': bucket_name,
            'S3Key': s3_key
        },
        Environment={
            'Variables': {
                'KNOWLEDGE_BASE_1_ID': Knowledge_Base_1_Id,
                'KNOWLEDGE_BASE_2_ID': Knowledge_Base_2_Id,
                'SYSTEM_FUNCTION_1_NAME': System_Function_1_Name,
                'SYSTEM_FUNCTION_2_NAME': System_Function_2_Name
            }
        },
        Timeout=60,
        MemorySize=512,
        Layers=[layer_arn],
        Description='Strands Agents Lambda function'
    )
    
    return response['FunctionArn']


def update_lambda_function_code(function_name, zip_file_path):
    client = boto3.client('lambda')
    with open(zip_file_path, 'rb') as f:
        zip_content = f.read()
    response = client.update_function_code(
        FunctionName=function_name,
        ZipFile=zip_content
    )
    return response
    
# Update or deploy Lambda function
if lambda_function_exists(function_name) == True:
    update_lambda_function_code(function_name, function_zip_path)
    print(f"Lambda code updated for function: {function_name}")
else:
    function_arn = deploy_lambda_function(function_zip_path, layer_arn, SolutionAccessRoleArn, CodeBucketForAutomationName)
    print(f"Lambda function deployed with ARN: {function_arn}")

# Wait for 10 seconds
print("Waiting for Lambda deployment to complete...")
time.sleep(10) 

# Check Lambda status
lambda_client = boto3.client('lambda')
response = lambda_client.get_function(FunctionName=function_name)
print(f"Lambda status: {response['Configuration']['State']}")

## Step 6: Test the Lambda Function

In [ ]:
def invoke_lambda_function():
    '''Invoke the Lambda function and return the response.'''
    lambda_client = boto3.client('lambda')
    
    payload = {
        'prompt': 'A new user is asking about the price of Doggy Delights?'
    }
    
    response = lambda_client.invoke(
        FunctionName=function_name,
        InvocationType='RequestResponse',
        Payload=json.dumps(payload)
    )
    
    response_payload = json.loads(response['Payload'].read().decode('utf-8'))
    return response_payload

# Invoke the Lambda function
response = invoke_lambda_function()
print("Lambda function response:")
print(json.dumps(response, indent=2))

## Step 7: Cleanup Resources (Optional)

In [ ]:
def cleanup_resources():
    '''Clean up AWS resources created in this notebook.'''
    lambda_client = boto3.client('lambda')
    s3_client = boto3.client('s3')
    
    try:
        # Get the function to retrieve layer information
        function_info = lambda_client.get_function(FunctionName=function_name)
        
        # Extract layer ARNs
        layer_arns = [layer['Arn'] for layer in function_info['Configuration'].get('Layers', [])]
        
        # Delete the Lambda function
        lambda_client.delete_function(FunctionName=function_name)
        print(f"Lambda function {function_name} deleted")
        
        # Delete each layer version
        for layer_arn in layer_arns:
            # Extract layer name and version from ARN
            parts = layer_arn.split(':')
            layer_name = parts[-2]
            layer_version = int(parts[-1])
            
            lambda_client.delete_layer_version(
                LayerName=layer_name,
                VersionNumber=layer_version
            )
            print(f"Layer {layer_name} version {layer_version} deleted")
        
        # Delete S3 objects
        s3_prefix = f"lambda-functions/{function_name}"
        response = s3_client.list_objects_v2(
            Bucket=CodeBucketForAutomationName,
            Prefix=s3_prefix
        )
        
        if 'Contents' in response:
            for obj in response['Contents']:
                s3_client.delete_object(
                    Bucket=CodeBucketForAutomationName,
                    Key=obj['Key']
                )
                print(f"S3 object {obj['Key']} deleted")
    
    except Exception as e:
        print(f"Error deleting lambda: {e}")
    
    # Clean up local files
    for file_path in [layer_zip_path, function_zip_path]:
        try:
            os.remove(file_path)
            print(f"Deleted local file: {file_path}")
        except Exception as e:
            print(f"Error deleting file: {e}")

# Uncomment the line below to clean up resources
# cleanup_resources()

## Summary

In this notebook, we demonstrated how to:

1. Use the agent directory structure for Lambda function code and requirements
2. Package Strands Agents SDK and its dependencies as a Lambda layer
3. Deploy the Lambda function and layer using boto3 with a provided IAM role
4. Test the deployed Lambda function